### 1. Understand einstem summation in numpy

Start with https://obilaniu6266h16.wordpress.com/2016/02/04/einstein-summation-in-numpy/ 

In [1]:
import numpy as np

In [4]:
# Example1:  of is a classic matrix mulitplication
# C = A * B
N_i = 2
N_k = 3
N_j = 4
A = np.random.rand(N_i, N_k)
B = np.random.rand(N_k, N_j)

## Method 1. Classic matrix multiplication
C = np.empty((N_i, N_j))
for i in range(N_i): # i and j are called free indices
    for j in range(N_j):
        total = 0
        for k in range(N_k): # k is called a summation index
            total += A[i,k] * B[k,j]
        C[i,j] = total

assert np.allclose(C, np.dot(A, B))

## Method 2. Einstein summation
C = np.einsum("ik, kj -> ij", A, B)
assert np.allclose(C, np.dot(A, B))

In [6]:
# Example2: extracting matrix diagonal of a square matrix
N = 5
A = np.random.rand(N, N)
d = np.empty(N)

## Method 1. Classic matrix diagonal extraction
for i in range(N): # free index i
    total = 0
    total += A[i,i] # summation index: None
    d[i] = total

assert np.allclose(d, np.diag(A))

## Method 2. Einstein summation
d = np.einsum("ii -> i", A)
assert np.allclose(d, np.diag(A))

In [8]:
# Example3: Matrix trace
N = 5
A = np.random.rand(N, N)

## Method 1. Classic matrix trace
total = 0
for i in range(N): # free index: None
    total += A[i][i] # summation index: i

assert np.allclose(total, np.trace(A))

## Method 2. Einsum notation
total = np.einsum("ii->", A)
assert np.allclose(total, np.trace(A))

In [18]:
# Example4: Quadratic form v^T @ A @ v
N = 5
v = np.random.rand(N)
A = np.random.rand(N,N)

## Method 1. Classic
out_sum = 0
for i in range(N): # free index: None
    total = 0
    for k in range(N): # summation index: i, k
        total += v[k] * A[k,i]

    out_sum += total * v[i]

assert np.allclose(out_sum, v.T @ A @ v)

## Method 2. einsum
out_sum = np.einsum("i,ij,j -> ", v.T, A, v)
assert np.allclose(out_sum, v.T @ A @ v)

In [22]:
# Example 5: Outer product of 2 vectors
Ni = 3
Nj = 5
v = np.random.rand(Ni)
z = np.random.rand(Nj)

## Method 1: v @ zT
A = np.empty((Ni, Nj))
for i in range(Ni):
    for j in range(Nj):
        A[i, j] = v[i] * z[j]

assert np.allclose(A, v.reshape(Ni,1) @ z.reshape(1, Nj))

## Method 2: einsum
A = np.einsum("i,j->ij", v, z)
assert np.allclose(A,  v.reshape(Ni,1) @ z.reshape(1, Nj))

In [30]:
# Example 6: Batched Outer product of 2 matrix
Batch = 3
Ni = 4
Nj = 5

A = np.random.rand(Batch, Ni)
B = np.random.rand(Batch, Nj)

# Method1 : class
C = np.empty((Batch, Ni, Nj))
for i in range(Batch): # i j k are all free indices
    for j in range(Ni):
        for k in range(Nj):
            C[i,j,k] = A[i,j] * B[i,k] # No summation index as we do not remove any index

# Method 2 : einsum
D = np.einsum("ij,ik->ijk", A, B)

assert np.allclose(C, D)

### 2. Reading introduction to numpy's einsum is really good as it has 3 concises points perfectly
https://ajcr.net/Basic-guide-to-einsum/

### 3. Einsum follow Tim rock
https://rockt.github.io/2018/04/30/einsum

In [2]:
import torch

In [3]:
# Matrix transpose
A = torch.arange(6).reshape(2,3)

print(A)
B = torch.einsum('ij->ji', A)
print(B)


tensor([[0, 1, 2],
        [3, 4, 5]])
tensor([[0, 3],
        [1, 4],
        [2, 5]])


In [4]:
# Sum
A = torch.arange(6).reshape(2,3)
print(A)
B = torch.einsum('ij->', A)
print(B)

tensor([[0, 1, 2],
        [3, 4, 5]])
tensor(15)


In [6]:
# Column sum
A = torch.arange(6).reshape(2,3)
print(A)
B = torch.einsum('ij->j', A)
print(B)

tensor([[0, 1, 2],
        [3, 4, 5]])
tensor([3, 5, 7])


In [10]:
# Dot product
a = torch.arange(3)
b = torch.arange(3)
c = torch.einsum('i,i->', a, b)
print(c)

# However, below expression is only doing sum each axis and multiply
# this is because different letter, thus they are not multiplied. But after finishing,
# they will be multiplied
c = torch.einsum('i,j->', a,b)

tensor(5)


In [11]:
# Outer product
a = torch.arange(3)
b = torch.arange(5)
c = torch.einsum('i,j->ij', a, b)
print(c)

tensor([[0, 0, 0, 0, 0],
        [0, 1, 2, 3, 4],
        [0, 2, 4, 6, 8]])


In [12]:
# Hadamard product
a = torch.arange(6).reshape(2,3)
b = torch.arange(6).reshape(2,3)
c = torch.einsum('ij,ij->ij', a, b)
print(c)

tensor([[ 0,  1,  4],
        [ 9, 16, 25]])


In [17]:
# Bilinear transformation
a = torch.randn(3,5)
b = torch.randn(1,5,6)
c = torch.randn(3,6)

d = torch.einsum('ik,jkl,il->ij', a,b,c)
print(d)

tensor([[-6.5276],
        [ 0.2850],
        [-0.7522]])
